In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import keras.backend as K
K.set_image_data_format("channels_last")
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import math
import h5py
import tensorflow as tf
from tensorflow.keras import callbacks


%matplotlib inline


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2 


# Read Metadata
meta_df = pd.read_csv(r"/content/drive/My Drive/fer2013.csv")

emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
classes = len(emotions)
meta_df = meta_df.rename(columns={"emotion":"label","pixels":"image"})
meta_df['emotion'] = meta_df['label'].apply(lambda x: emotions[int(x)])
meta_df = meta_df.drop(columns=['Usage'])
print(classes)

7


In [0]:
meta_df 

,label,image,emotion
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Angry
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Angry
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Fear
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Sad
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Neutral
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,Neutral
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,Happy
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,Angry
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,Happy


In [0]:
train_set_x_orig=[]
for id in range(35887):
    image = np.reshape(np.array(meta_df.image[id].split(' ')).astype(int),(48,48, 1))
    train_set_x_orig.append(image)
train_set_x_orig = np.array(train_set_x_orig)
print(train_set_x_orig.shape)

(35887, 48, 48, 1)


In [0]:
train_set_y_orig = np.array(meta_df.label)
print(train_set_y_orig.shape[0])

35887


In [0]:
test_set_x_orig = []
test_set_y_orig = []
test_set_x_orig = np.array(test_set_x_orig)
test_set_y_orig = np.array(test_set_y_orig)

In [0]:
from sklearn.model_selection import train_test_split
train_set_x_orig, test_set_x_orig, train_set_y_orig, test_set_y_orig = train_test_split(train_set_x_orig, train_set_y_orig,shuffle = True, stratify = train_set_y_orig, random_state = 2020, test_size = 0.2 )

print(train_set_x_orig.shape)
print(test_set_x_orig.shape)
print(train_set_y_orig.shape)
print(test_set_x_orig.shape)

(28709, 48, 48, 1)
(7178, 48, 48, 1)
(28709,)
(7178, 48, 48, 1)


In [0]:
# Normalize image vectors
X_train = train_set_x_orig/255.
X_test = test_set_x_orig/255.

# Reshape
Y_train = train_set_y_orig.T
Y_test = test_set_y_orig.T


print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 28709
number of test examples = 7178
X_train shape: (28709, 48, 48, 1)
Y_train shape: (28709,)
X_test shape: (7178, 48, 48, 1)
Y_test shape: (7178,)


In [0]:
from keras.utils import to_categorical
Y_train = to_categorical(Y_train, num_classes = classes)



In [0]:
Y_test = to_categorical(Y_test, num_classes = classes)
print(Y_train.shape)
print(Y_test.shape)

(28709, 7)
(7178, 7)


In [0]:
def Moodel(input_shape):
   
    
    X_input = Input(input_shape)
    
    #Block 1 -- conv2d - MaxPooling
    
    X = ZeroPadding2D((3,3))(X_input)
    
    X = Conv2D(32,(3,3), strides = (1,1), name = 'conv0')(X)
    X = BatchNormalization(axis=3, name = 'bn0')(X)
    X = Activation('relu')(X)
    
    #MaxPool
    
    X = MaxPooling2D((3,3), strides = (1,1), name = 'max_pool_0')(X)
    
    
    #Block2 -- conv2D -- maxpool2D
    
    X = Conv2D(64,(3,3), name = 'conv1')(X)
    X = BatchNormalization(axis=3, name = 'bn1')(X)
    X = Activation('relu')(X)
    
    #MaxPool
    
    X= MaxPooling2D((2,2), strides = (2,2), name = 'max_pool_1')(X)
    
    
    #Block3 -- conv2D -- MaxPool2D
    
    X = Conv2D(128,(3,3), name ="conv2")(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation("relu")(X)
    
    #MaxPool
    
    X = MaxPooling2D((3,3), strides = (1,1), name = 'max_pool_2')(X)
    
    
    #Block4 -- conv2D -- MaxPool2D
    
    X = Conv2D(256,(3,3), name ="conv3")(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation("relu")(X)
    
    #MaxPool
    
    X = MaxPooling2D((2,2), strides = (2,2), name = 'max_pool_3')(X)
    
    
    #Block5 -- conv2D -- MaxPool2D
    
    X = Conv2D(512,(3,3), name ="conv4")(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation("relu")(X)
    
    #MaxPool
    
    X = MaxPooling2D((2,2), strides = (2,2), name = 'max_pool_4')(X)
    
    #Block6 --flatten --dense
    
    X = layers.Dropout(0.5)(X)
    X = Flatten()(X)
    X = Dense(128, activation = 'relu', name = 'fc')(X)
    X = Dense(64, activation = 'relu', name = 'fc1')(X)
    X = Dense(7, activation = 'softmax', name = 'fc2')(X)
    
   
    
     #Block5 
    
    model = Model(inputs = X_input, output = X, name = 'Moodel')
    
    return model

In [0]:
moodel = Moodel(X_train.shape[1:])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., name="Moodel", outputs=Tensor("fc...)`


In [0]:
my_callback = [tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 4)]

In [0]:
moodel.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

AttributeError: ignored

In [0]:
history = moodel.fit(X_train, Y_train, epochs = 60, batch_size=200, verbose = 1, validation_split = 0.2, callbacks = my_callback)


Train on 22967 samples, validate on 5742 samples
Epoch 1/60
22967/22967 [==============================] - 12s 534us/step - loss: 1.7111 - accuracy: 0.3421 - val_loss: 1.8610 - val_accuracy: 0.1722
Epoch 2/60
22967/22967 [==============================] - 11s 492us/step - loss: 1.3513 - accuracy: 0.4850 - val_loss: 1.8524 - val_accuracy: 0.1999
Epoch 3/60
22967/22967 [==============================] - 11s 492us/step - loss: 1.2321 - accuracy: 0.5302 - val_loss: 1.5845 - val_accuracy: 0.3802
Epoch 4/60
22967/22967 [==============================] - 11s 494us/step - loss: 1.1427 - accuracy: 0.5676 - val_loss: 1.4586 - val_accuracy: 0.4624
Epoch 5/60
22967/22967 [==============================] - 11s 494us/step - loss: 1.0867 - accuracy: 0.5895 - val_loss: 1.1851 - val_accuracy: 0.5488
Epoch 6/60
22967/22967 [==============================] - 11s 492us/step - loss: 1.0430 - accuracy: 0.6088 - val_loss: 1.1405 - val_accuracy: 0.5688
Epoch 7/60
22967/22967 [==============================] -

In [0]:

moodel.evaluate(X_test,Y_test)

7178/7178 [==============================] - 2s 219us/step


[2.4720780860085645, 0.6023961901664734]

In [0]:

y_proba = moodel.predict(X_test)


In [0]:
print(y_proba)

[[1.0350620e-02 5.9492182e-11 3.0311728e-10 ... 1.1485775e-08
  1.2241052e-05 1.2422082e-04]
 [1.1646414e-04 1.4351420e-10 2.0058017e-04 ... 1.0236659e-01
  7.6011027e-08 8.9731622e-01]
 [9.8489064e-01 6.0296884e-06 1.6410068e-03 ... 1.3458175e-02
  9.9432805e-07 2.4383596e-06]
 ...
 [7.2114331e-01 1.8799008e-06 2.5575882e-01 ... 1.2899247e-07
  1.9379528e-05 2.3059485e-02]
 [5.1630830e-04 1.4630030e-03 5.9978972e-04 ... 5.8378141e-08
  9.9742091e-01 3.3950873e-09]
 [7.5959754e-01 3.4040813e-06 5.1175494e-04 ... 1.6808933e-01
  1.0458949e-04 2.0464380e-05]]


In [0]:
#preds = keras.utils.np_utils.probas_to_classes(y_proba)
#print ("Loss = " + str(preds[0]))
#print ("Test Accuracy = " + str(preds[1]))

In [0]:
moodel.save(r"/content/drive/My Drive/Moodel1")